In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
try:
    import cPickle as pickle
except ImportError:
    import _pickle as pickle
import matplotlib.pyplot as plt

In [5]:
df_orig = pd.read_csv('../data/MKdataset07Feb17.csv')
df_orig.head()

,bloc,icustayid,charttime,gender,age,elixhauser,re_admission,died_in_hosp,mortality_90d,Weight_kg,...,mechvent,SOFA,SIRS,median_dose_vaso,max_dose_vaso,input_total_tev,input_4hourly_tev,output_total,output_4hourly,cumulated_balance_tev
0,1,3,7245052800,0,17639.826435,0.0,0,0,1,77.5,...,0,9,4,0.0,0.0,3500.0,2100.0,230.0,230.0,3270.0
1,2,3,7245067200,0,17639.826435,0.0,0,0,1,77.5,...,0,8,3,0.0,0.0,5499.0,1999.0,697.0,467.0,4802.0
2,3,3,7245081600,0,17639.826435,0.0,0,0,1,77.5,...,0,7,3,0.0,0.0,5604.0,105.0,2302.0,1605.0,3302.0
3,4,3,7245096000,0,17639.826435,0.0,0,0,1,77.5,...,0,6,3,0.0,0.0,5664.0,60.0,2922.0,620.0,2742.0
4,5,3,7245110400,0,17639.826435,0.0,0,0,1,77.5,...,0,5,3,0.0,0.0,5724.0,60.0,3352.0,430.0,2372.0


In [24]:
# Number of records in original dataset: 242450
# Number of unique ICU ID's: 17898
num_icuid = df_orig.icustayid.nunique()

In [6]:
df_orig.columns

Index(['bloc', 'icustayid', 'charttime', 'gender', 'age', 'elixhauser',
       're_admission', 'died_in_hosp', 'mortality_90d', 'Weight_kg', 'GCS',
       'HR', 'SysBP', 'MeanBP', 'DiaBP', 'Shock_Index', 'RR', 'SpO2', 'Temp_C',
       'FiO2_1', 'Potassium', 'Sodium', 'Chloride', 'Glucose', 'BUN',
       'Creatinine', 'Magnesium', 'Calcium', 'Ionised_Ca', 'CO2_mEqL', 'SGOT',
       'SGPT', 'Total_bili', 'Albumin', 'Hb', 'WBC_count', 'Platelets_count',
       'PTT', 'PT', 'INR', 'Arterial_pH', 'paO2', 'paCO2', 'Arterial_BE',
       'Arterial_lactate', 'HCO3', 'PaO2_FiO2', 'mechvent', 'SOFA', 'SIRS',
       'median_dose_vaso', 'max_dose_vaso', 'input_total_tev',
       'input_4hourly_tev', 'output_total', 'output_4hourly',
       'cumulated_balance_tev'],
      dtype='object')

In [32]:
aggregated_by_icuid = df_orig.groupby('icustayid').agg(num_unique = ("charttime", "nunique"), 
                                 start_time = ( "charttime", "min"), 
                                 end_time = ( "charttime", "max"), 
                                 mean_elixhauser = ('elixhauser', 'mean'),
                                 age = ('age', 'mean'),
                                 died_in_hosp = ('died_in_hosp', 'mean'),
                                 mortality_90d = ('mortality_90d', 'mean')
                                
                                ).reset_index() 
aggregated_by_icuid['num_days'] = (aggregated_by_icuid['end_time'] - aggregated_by_icuid['start_time'])/(3600*24)
aggregated_by_icuid['age'] /= 365
aggregated_by_icuid.describe()

,icustayid,num_unique,start_time,end_time,mean_elixhauser,age,died_in_hosp,mortality_90d,num_days
count,17898.000000,17898.000000,1.789800e+04,1.789800e+04,17898.000000,17898.000000,17898.000000,17898.000000,17898.000000
mean,50204.658286,13.546206,5.728567e+09,5.728760e+09,3.797855,64.727524,0.129344,0.225779,2.238639
std,28929.843021,4.209456,9.194877e+08,9.194877e+08,2.126465,16.904665,0.335590,0.418106,0.625090
min,3.000000,1.000000,4.116047e+09,4.116205e+09,0.000000,18.034034,0.000000,0.000000,0.000000
25%,24820.750000,10.000000,4.944942e+09,4.945122e+09,2.000000,53.633529,0.000000,0.000000,1.833333
50%,50554.000000,14.000000,5.727298e+09,5.727506e+09,4.000000,66.168829,0.000000,0.000000,2.166667
75%,75143.500000,16.000000,6.531012e+09,6.531278e+09,5.000000,78.195957,0.000000,0.000000,2.666667
max,99995.000000,20.000000,7.560386e+09,7.560574e+09,14.000000,91.462603,1.000000,1.000000,3.166667


In [26]:
len(aggregated_by_icuid[aggregated_by_icuid['died_in_hosp'] == 1])/num_icuid

0.12934406078891497

In [9]:
df_orig['diff'] = df_orig['charttime'].diff()

s = df_orig.groupby('icustayid').agg(diff_count = ('diff', 'nunique')).reset_index()
df_orig['diff'].value_counts()

 1.440000e+04    216018
 2.880000e+04      3880
 4.320000e+04      2599
 5.760000e+04      1505
 7.200000e+04       494
                  ...  
-1.406185e+09         1
-3.930443e+08         1
 7.030717e+08         1
 1.943002e+09         1
-9.223372e+18         1
Name: diff, Length: 17896, dtype: int64

In [10]:
df_orig[df_orig['icustayid'] == 20962][['icustayid', 'charttime', 'diff', 'input_4hourly_tev', 'input_total_tev', 'output_total', 'output_4hourly']]


,icustayid,charttime,diff,input_4hourly_tev,input_total_tev,output_total,output_4hourly
51056,20962,5979585000,252048120.0,1523.249767,19683.119550,5106.0,0.0
51057,20962,5979599400,14400.0,1150.195583,20833.315133,5106.0,0.0
51058,20962,5979613800,14400.0,166.000000,20999.315133,5106.0,0.0
51059,20962,5979657000,43200.0,367.337650,21366.652783,5106.0,0.0
51060,20962,5979685800,28800.0,494.214750,21860.867533,5106.0,0.0
51061,20962,5979743400,57600.0,450.445333,22311.312867,5106.0,0.0
51062,20962,5979757800,14400.0,10.012000,22321.324867,5106.0,0.0
51063,20962,5979844200,86400.0,80.052000,22401.376867,5106.0,0.0


In [33]:
columns = ['median_dose_vaso', 'max_dose_vaso', 'input_total_tev', 'input_4hourly_tev', 'output_total', 'output_4hourly', 'cumulated_balance_tev']
fluids_df = df_orig[columns]
fluids_df.describe()

,median_dose_vaso,max_dose_vaso,input_total_tev,input_4hourly_tev,output_total,output_4hourly,cumulated_balance_tev
count,242450.000000,242450.000000,242450.000000,242450.000000,242450.000000,242450.000000,242450.000000
mean,0.060122,0.073422,6948.611820,325.125317,4159.768038,308.758480,2786.388356
std,0.244487,0.299033,8771.074609,601.380127,6993.859031,349.829526,8200.366224
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-188596.125350
25%,0.000000,0.000000,1741.162879,9.583333,690.000000,75.000000,-406.366900
50%,0.000000,0.000000,4650.000000,80.000000,2085.000000,215.000000,1675.888208
75%,0.000000,0.000000,9028.000000,400.000000,4740.000000,420.000000,5187.165413
max,6.703000,10.300000,315802.821083,17058.576800,231820.000000,7600.000000,313325.996083
